# Выбор локации для скважины

## Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.
Шаги для выбора локации обычно такие:
В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
Строят модель для предсказания объёма запасов в новых скважинах;
Выбирают скважины с самыми высокими оценками значений;
Определяют регион с максимальной суммарной прибылью отобранных скважин.
Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.



## Инструкция по выполнению проекта

- Загрузите и подготовьте данные. Поясните порядок действий.
- Обучите и проверьте модель для каждого региона:
    -  Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
    -  Обучите модель и сделайте предсказания на валидационной выборке.
    -  Сохраните предсказания и правильные ответы на валидационной выборке.
    -  Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
    -  Проанализируйте результаты.

- Подготовьтесь к расчёту прибыли:
    -  Все ключевые значения для расчётов сохраните в отдельных переменных.
    -  Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
    -  Напишите выводы по этапу подготовки расчёта прибыли.

- Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
    -  Выберите скважины с максимальными значениями предсказаний.
    -  Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
    -  Рассчитайте прибыль для полученного объёма сырья.

- Посчитайте риски и прибыль для каждого региона:
    -  Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
    -  Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
    -  Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

**Описание данных**

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

**Условия задачи:**
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.
- Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

# Выполнение работы

### Первичная визуальная оценка данных

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error 
from scipy import stats as st

from IPython.display import display_html

In [2]:
# Импортируем данные
try :
    geo_0 = pd.read_csv('C:/Users/Админ/Desktop/Jupyter/Yandex/geo_data_0.csv')
    geo_1 = pd.read_csv('C:/Users/Админ/Desktop/Jupyter/Yandex/geo_data_1.csv')
    geo_2 = pd.read_csv('C:/Users/Админ/Desktop/Jupyter/Yandex/geo_data_2.csv')

except:
    geo_0 = pd.read_csv('/Desktop/Jupyter/Yandex/geo_data_0.csv')
    geo_1 = pd.read_csv('/Desktop/Jupyter/Yandex/geo_data_1.csv')
    geo_2 = pd.read_csv('/Desktop/Jupyter/Yandex/geo_data_2.csv')
    

In [3]:
# Выполняем визуальную оценку данных

print('ПЕРВЫЕ СТРОКИ ТАБЛИЦ')
    print('Регион 0'), print(geo_0.head()) 
    print(' ')
    print('Регион 1'), print(geo_1.head()) 
    print(' ')
    print('Регион 2'),print(geo_2.head())
    print(' ')
    print('ОБЩАЯ ИНФОРМАЦИЯ')
    print('Регион 0'), print(geo_0.info()) 
    print(' ')
    print('Регион 1'), print(geo_1.info()) 
    print(' ')
    print('Регион 2'),print(geo_2.info())
    print(' ')
    print('КОЛИЧЕСТВО ПРОПУСКОВ')
    print('Регион 0'), print(geo_0.isna().sum()) 
    print(' ')
    print('Регион 1'), print(geo_1.isna().sum()) 
    print(' ')
    print('Регион 2'),print(geo_2.isna().sum())


ПЕРВЫЕ СТРОКИ ТАБЛИЦ
Регион 0
      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
 
Регион 1
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
 
Регион 2
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746
 
ОБЩАЯ ИНФОРМАЦИЯ

(None, None)

In [4]:
    print('КОЛИЧЕСТВО ДУБЛИКАТОВ')
    print('Регион 0'),print(geo_0[geo_0.duplicated(subset = 'id', keep = False)].sort_values('id')) 
    print(' ')
    print('Регион 1'),print(geo_1[geo_1.duplicated(subset = 'id', keep = False)].sort_values('id')) 
    print(' ')
    print('Регион 2'),print(geo_2[geo_2.duplicated(subset = 'id', keep = False)].sort_values('id'))
    print(' ')
    print('КОЛИЧЕСТВО ДУБЛИКАТОВ id')
    print('Регион 0'),print(geo_0['id'].duplicated().sum(),'; Процент пропусков:',geo_0['id'].duplicated().sum()/len(geo_2)*100) 
    print(' ')
    print('Регион 1'),print(geo_1['id'].duplicated().sum(),'; Процент пропусков:',geo_1['id'].duplicated().sum()/len(geo_2)*100) 
    print(' ')
    print('Регион 2'),print(geo_2['id'].duplicated().sum(), '; Процент пропусков:',geo_2['id'].duplicated().sum()/len(geo_2)*100)

КОЛИЧЕСТВО ДУБЛИКАТОВ
Регион 0
          id        f0        f1         f2     product
66136  74z30  1.084962 -0.312358   6.990771  127.643327
64022  74z30  0.741456  0.459229   5.153109  140.771492
51970  A5aEY -0.180335  0.935548  -2.094773   33.020205
3389   A5aEY -0.039949  0.156872   0.209861   89.249364
69163  AGS9W -0.933795  0.116194  -3.655896   19.230453
42529  AGS9W  1.454747 -0.479651   0.683380  126.370504
931    HZww2  0.755284  0.368511   1.863211   30.681774
7530   HZww2  1.061194 -0.373969  10.430210  158.828695
63593  QcMuo  0.635635 -0.473422   0.862670   64.578675
1949   QcMuo  0.506563 -0.323775  -2.215583   75.496502
75715  Tdehs  0.112079  0.430296   3.218993   60.964018
21426  Tdehs  0.829407  0.298807  -0.049563   96.035308
92341  TtcGQ  0.110711  1.022689   0.911381  101.318008
60140  TtcGQ  0.569276 -0.104876   6.440215   85.350186
89582  bsk9y  0.398908 -0.400253  10.122376  163.433078
97785  bsk9y  0.378429  0.005837   0.160827  160.637302
41724  bxg6G -0.8

(None, None)

In [5]:
    print('ОПИСАНИЕ ДАННЫХ')
    print('Регион 0'),print(geo_0.describe()) 
    print(' ')
    print('Регион 1'),print(geo_1.describe()) 
    print(' ')
    print('Регион 2'),print(geo_2.describe())    
    print(' ')
    print('ОБЪЕМ ЗАПАСОВ СКВАЖИНЫ')
    print('Регион 0'),print(geo_0['product'].sum().round(1), ' ; среднее = ', geo_0['product'].mean()) 
    print(' ')
    print('Регион 1'),print(geo_1['product'].sum().round(1), ' ; среднее = ', geo_1['product'].mean())
    print(' ')
    print('Регион 2'),print(geo_2['product'].sum().round(1), ' ; среднее = ', geo_2['product'].mean())

ОПИСАНИЕ ДАННЫХ
Регион 0
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
 
Регион 1
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985     

(None, None)

In [6]:
def display_side_by_side(*args):
    html_str=' '
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [7]:
def corr_region(data):
    return data.corr().style.background_gradient(cmap = 'Blues')

In [8]:
print('КОРРРЕЛЯЦИЯ РЕГИОНОВ: РЕГИОН 0, РЕГИОН 1, РЕГИОН 2')
display_side_by_side(corr_region(geo_0),corr_region(geo_1),corr_region(geo_2))

КОРРРЕЛЯЦИЯ РЕГИОНОВ: РЕГИОН 0, РЕГИОН 1, РЕГИОН 2


,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000
,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000
,f0,f1,f2,product


### Вывод 

- В 3х регионах одинаковое количество данных: 100 000 строк
- Типы данных не требуют изменений
- В данных отсутвуют пропуски
- В данных имеются дубликаты id :
    - Регион 0 = 10
    - Регион 1 = 4
    - Регион 2 = 4

Причина появления дубликатов не известна, возможно замер показателей скважины производился 2 раза. Данные дубликаты можно оставить, так как их наличие или отсутсвие не окажет существенного влияния на ход исследования - их процент от исхоных данных составляет менее 0.5%

- По объему запасов в скважине лидирует:
    - Регион 2 = 9 500 000.0 тыс. баррелей
    - Регион 0 = 9 250 000.0 тыс. баррелей
    - Регион 1 = 6 882 500.0 тыс. баррелей
    
- Во всех регионах, из всех признаков, наибольшую корреляцию с целевым признаком Products имеет признак f2. Наиболее выраженная коррекляция наблюдается в Регионе 1 - корреляция с f2 = 0.999

### Построение о обучение моделей

1. - Обучите и проверьте модель для каждого региона:
    -  Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
    -  Обучите модель и сделайте предсказания на валидационной выборке.
    -  Сохраните предсказания и правильные ответы на валидационной выборке.
    -  Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
    -  Проанализируйте результаты.

**Подготовка выборок**

In [9]:
# Разбиение данных на даннные из целевого и общих признаков:

def features_definition(df):

    features = df.drop(['id','product'], axis=1)
    target = df['product']
    
    return features,target

features_0,target_0 = features_definition(geo_0)
features_1,target_1 = features_definition(geo_1)
features_2,target_2 = features_definition(geo_2)

In [10]:
# Разбиение данных на обучающие, валидационные и тестовые выборки:

def sep_train_valid_test(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, train_size=0.75,random_state=12345)  # разделение на тренировочный сет и валидационный    
    return features_train, target_train, features_valid, target_valid

features_train_0, target_train_0, features_valid_0, target_valid_0 = sep_train_valid_test(features_0, target_0)
features_train_1, target_train_1, features_valid_1, target_valid_1 = sep_train_valid_test(features_1,target_1)
features_train_2, target_train_2, features_valid_2, target_valid_2 = sep_train_valid_test(features_2,target_2)

print('Размер обучающей выборки равен:', features_train_0.shape)
print('Размер валидационной выборки равен:', features_valid_0.shape)

Размер обучающей выборки равен: (75000, 3)
Размер валидационной выборки равен: (25000, 3)


**1. - Обучение и проверка моделей для каждого региона:**

In [11]:
# Создадим функцию для обучения модели:

def model_fit(features_train, target_train, features_valid, target_valid, region):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result_rmse = mean_squared_error(target_valid, predictions_valid)**0.5
    print('Cредний запас предсказанного сырья по ', region, predictions_valid.mean())
    print('RMSE модели по ', region, result_rmse)
    return model, predictions_valid, result_rmse

In [12]:
model_0, predictions_valid_0, result_rmse_0 = model_fit(features_train_0, target_train_0, features_valid_0, target_valid_0, 'Регион 0:')

Cредний запас предсказанного сырья по  Регион 0: 92.59256778438035
RMSE модели по  Регион 0: 37.5794217150813


In [13]:
model_1, predictions_valid_1, result_rmse_1 = model_fit(features_train_1, target_train_1, features_valid_1, target_valid_1, 'Регион 1:')

Cредний запас предсказанного сырья по  Регион 1: 68.72854689544602
RMSE модели по  Регион 1: 0.8930992867756168


In [14]:
model_2, predictions_valid_2, result_rmse_2 = model_fit(features_train_2, target_train_2, features_valid_2, target_valid_2, 'Регион 2:')

Cредний запас предсказанного сырья по  Регион 2: 94.96504596800489
RMSE модели по  Регион 2: 40.02970873393434


**Вывод**

Распределение регионов по запасам сырья:
 - Регион 2 = 94.9 тыс.баррелей
 - Регион 0 = 92.6 тыс. баррелей
 - Регион 1 = 68.7 тыс.баррелей
Получившиеся предсказанные данные о запасах чрезвычайно близки к фактическим среним значениям, полученным при первичной визуальной оценке данных


### Подготовка и расчет прибыли

2. - Подготовьтесь к расчёту прибыли:
    -  Все ключевые значения для расчётов сохраните в отдельных переменных.
    -  Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
    -  Напишите выводы по этапу подготовки расчёта прибыли.

In [15]:
# Зададим ключевые (константные) значения, указанные по условию задачи:

BUDGET_ON_REGION = 10000000000                      # Бюджет на разработку скважин в регионе
WELLS_COUNT = 500                                   # Количество скажин для исследования
BEST_WELLS = 200                                    # Количество скважин в регионе для продолжения разработки
REVENUE_PER_BAR = 450000                            # Доход с тысячи баррелей

**Расчет точки безубыточности**

Для расчета достаточного объема сырья в скважине для безубыточной разработки новой скважины, необходимо поделить планируемый объем инвестиций в одну из 200 лучших скважин, на доход с 1ой тысячи баррелей: 

- Инвестиции на скважину = Бюджет на разработку в регионе(BUDGET_ON_REGION) / количество скважин(BEST_WELLS)
- Объем сырья = Инвестиции на скважину / REVENUE_PER_BAR


In [16]:
# Расчитаем обем сырья, необходимый для безубыточной разработки скважины:

BUDGET_ON_BEST_WELLS = BUDGET_ON_REGION/BEST_WELLS 

OIL_VALUE = BUDGET_ON_BEST_WELLS/REVENUE_PER_BAR
print(f'Объем сырья, требующийся для безубыточной разработки скважины: {OIL_VALUE:.2f}')

Объем сырья, требующийся для безубыточной разработки скважины: 111.11


In [17]:
# Сравним полученный объем с предсказанным объемом средним объемом в регионе
def compare(predictions_valid, region):
    compare = (OIL_VALUE/predictions_valid.mean() * 100 - 100).round(1)
    print(f"Запас сырья в Регионе {region} меньше требуемого объема на: {compare}%")
    return

compare(predictions_valid_0, 0)
compare(predictions_valid_1, 1)
compare(predictions_valid_2, 2)

Запас сырья в Регионе 0 меньше требуемого объема на: 20.0%
Запас сырья в Регионе 1 меньше требуемого объема на: 61.7%
Запас сырья в Регионе 2 меньше требуемого объема на: 17.0%


**Вывод:**
Объем сырья, необходимый для безубыточной разработки скважин, превышает имеющийся средний запас сырья по регионам. Ни один из регионов предварительно не может удовлетворить условию безубыточности. Однако, в Регионах 0 и 2 в данных product 75% квантиль находится на уровне 128 и 130 тыс.баррелей, а в Регионе 1 максимальное значение product = 137 тыс.баррелей соответственно. Это значит, что несмотря на то, что среднее значение по Регионам не удовлетворяет условию безубыточности, во всех 3х регионах все же есть скважины, пригодные для безубыточной разработки.

**Выделение скважин, удовлетворяющих условию безубыточности**

In [18]:
def relevant_wells(data, region):
    counter = len(data[data['product'] > OIL_VALUE])
    percent = counter/len(data['product'])
    print(f'Количество скважин, удовлетворящих условию безубыточности в {region} = {counter}')
    print(f'Процент скважин, удовлетворящих условию безубыточности в {region} = {percent:.2%}')
    print(" ")
    return counter, percent

In [19]:
relevant_wells(geo_0, 'Регион 0')
relevant_wells(geo_1, 'Регион 1')
relevant_wells(geo_2, 'Регион 2')

Количество скважин, удовлетворящих условию безубыточности в Регион 0 = 36583
Процент скважин, удовлетворящих условию безубыточности в Регион 0 = 36.58%
 
Количество скважин, удовлетворящих условию безубыточности в Регион 1 = 16537
Процент скважин, удовлетворящих условию безубыточности в Регион 1 = 16.54%
 
Количество скважин, удовлетворящих условию безубыточности в Регион 2 = 38178
Процент скважин, удовлетворящих условию безубыточности в Регион 2 = 38.18%
 


(38178, 0.38178)

**Вывод:**

**Все рассматриваемые регионы имеют достаточный запас скважин для безубыточной разработки.**

### Расчет прибыли по скважинам в регионах

3. - Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:
    -  Выберите скважины с максимальными значениями предсказаний.
    -  Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
    -  Рассчитайте прибыль для полученного объёма сырья.

In [20]:
# Предсказанный по валидационной выборке объем сырья в скважинах преобразуем в Series:

predictions_valid_df_0 = pd.Series(predictions_valid_0,index=target_valid_0.index)
predictions_valid_df_1 = pd.Series(predictions_valid_1,index=target_valid_1.index)
predictions_valid_df_2 = pd.Series(predictions_valid_2,index=target_valid_2.index)

In [21]:
# Создадим функцию, которая выбирает 200 скважин с максимальным объемом сырья из Series предсказанных значений, 
# и вычисляет выручку по данным скважинам

def revenue_calculate(target, predictions):
    revenue = 0 
    data = pd.concat([predictions, target],axis=1) # - создаем дата-сет из предсказанных объемов и фактических
    data.columns = ['predictions','target'] # - задаем название столбцам
    
    best_wells_df = data.sort_values(by = 'predictions',ascending=False).head(BEST_WELLS) # Сортируем данные в порядке убывания по предсказанным значениям
    selected_wells = target[best_wells_df.index][:BEST_WELLS] # Отбираем целевые значения объемов сырья в количестве 200шт.
    
    revenue = ((selected_wells - OIL_VALUE) * REVENUE_PER_BAR).sum() # Расчитываем доход, полученный после преодоление точки безубыточности
    return revenue   

### Расчет рисков и прибыли для каждого региона

4. - Посчитайте риски и прибыль для каждого региона:
    -  Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
    -  Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
    -  Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

In [22]:
def profit_and_risks(target, probabilities, region):
        
    state = np.random.RandomState(12345)
    values = []        
        
    for i in range(1000):
        target_subsample = target.sample(n = WELLS_COUNT, replace=True, random_state=state) 
        probs_subsample = probabilities[target_subsample.index]
        revenue = revenue_calculate(target_subsample, probs_subsample)
        values.append(revenue)
    
    values = pd.Series(values)
    lower = values.quantile(0.05)/10e6
    upper = values.quantile(0.95)/10e6
    mean = values.mean()/10e6
    
    risks = (values < 0).mean()
    
    print(f'Средняя выручка в регионе {region}: {mean:.2f} млн.руб')
    print(f'95% доверительный интервал в регионе {region}: от {lower:.2f} до {upper:.2f}  млн.руб')
    print(f'Вероятность убытков в регионе {region}: {risks:.2%}')
    print("")
                       
    return mean,lower,upper,risks

In [23]:
mean_0,lower_0,upper_0,risks_0 = profit_and_risks(target_valid_0, predictions_valid_df_0, 0)
mean_1,lower_1,upper_1,risks_1 = profit_and_risks(target_valid_1, predictions_valid_df_1, 1)
mean_2,lower_2,upper_2,risks_2 = profit_and_risks(target_valid_2, predictions_valid_df_2, 2)

Средняя выручка в регионе 0: 42.59 млн.руб
95% доверительный интервал в регионе 0: от -3.18 до 88.13  млн.руб
Вероятность убытков в регионе 0: 6.00%

Средняя выручка в регионе 1: 51.52 млн.руб
95% доверительный интервал в регионе 1: от 15.08 до 86.15  млн.руб
Вероятность убытков в регионе 1: 1.00%

Средняя выручка в регионе 2: 43.50 млн.руб
95% доверительный интервал в регионе 2: от -4.34 до 89.80  млн.руб
Вероятность убытков в регионе 2: 6.40%



## Вывод

Из 3х регионов требованию о допустимой вероятности убытка в 2,5% соответвует лишь Регион 1. 

- Средняя выручка в регионе 1: 51.52
- 95% доверительный интервал в регионе 1: от 15.08 до 86.15
- Вероятность убытков в регионе 1: 1.00%

Для решения данной задачи была построенная модель машинного обучения Линейная Регрессия. Результаты предсказаний данной модели:
Распределение регионов по запасам сырья:
 - Регион 2 = 94.9 тыс.баррелей
 - Регион 0 = 92.6 тыс. баррелей
 - Регион 1 = 68.7 тыс.баррелей
Получившиеся предсказанные данные о запасах чрезвычайно близки к фактическим среним значениям, полученным при первичной визуальной оценке данных. Можно сделать вывод о высокой точности построенной модели. 

Однако данные RMSE по данным регионам: 
- RMSE модели по региону Регион 0: 37.579
- RMSE модели по региону Регион 1: 0.893
- RMSE модели по региону Регион 2: 40.029
